In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Normalization
from sklearn.preprocessing import StandardScaler

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')

dataset.head()

In [ ]:
# Manipulate sex, smoker, region
dataset["sex"] = dataset["sex"].str.strip().map({"male": 1, "female": 0})
dataset["smoker"] = dataset["smoker"].str.strip().map({"yes": 1, "no": 0})
dataset['region'] = dataset['region'].astype('category').cat.codes

dataset.head()

In [ ]:
test_dataset = dataset.sample(frac=0.2, random_state=42)
train_dataset = dataset.drop(test_dataset.index)

test_labels = test_dataset.pop("expenses")
train_labels = train_dataset.pop("expenses")

In [ ]:
len(test_dataset), len(train_dataset)

In [ ]:
normalizer = Normalization()
normalizer.adapt(np.array(train_dataset))

model = Sequential([
    normalizer,
    Dense(64, activation="relu"),
    Dense(64, activation="relu"),
    Dense(1)
])
model.compile(optimizer="adam", loss="mae", metrics=["mae", "mse"])
model.fit(train_dataset, train_labels, epochs=100, validation_split=0.2)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
